<a href="https://colab.research.google.com/github/oceam/agri-image/blob/main/codes/Googlecolab_fundamentals_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Googlecolab_fundamentals_5

Extract plant(green) from Image

Wei Guo  
2022.08.26->2023.08.30


## Prepare the data

In [ ]:
import gdown
# Download the data you pre-procssed
#https://drive.google.com/file/d/1grhJZR1bjsyOwvHCa-2NQ8iwnOfQJyyA/view?usp=sharing
url = "https://drive.google.com/file/d/1tubZ1K5Ku-lv3kieo68PnxjmOSkFAwNn/view?usp=sharing"
output3= "soybean_s_c.zip"
gdown.download(url=url, output=output3, quiet=False, fuzzy=True)

In [ ]:
# unzip image
!unzip soybean_s_c.zip

In [ ]:
from skimage import io
import os
imgname= os.path.join('content/soybean_s_c', 'C5_086_c.JPG')
I=io.imread(imgname)
io.imshow(I)

## Segmentation use color index values


In the paper of "[Verification of color vegetation indices for automated crop imaging applications](https://doi.org/10.1016/j.compag.2008.03.009)
", Meyer proposed three index calculated from RGB values: `NDI`, `ExG`, and `ExR`

![](https://www.dropbox.com/s/4dyiju0vg0bnmkz/colab_20210629173055.png?dl=1)

By using three values, it strength the differences between leaves and backgrounds.

Meanwhile, instead of manual given a value (e.g. 100 in previous cases, it used an automatic method: Otsu Threshold)

<p><a href="https://commons.wikimedia.org/wiki/File:Otsu%27s_Method_Visualization.gif#/media/File:Otsu's_Method_Visualization.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/3/34/Otsu%27s_Method_Visualization.gif" alt="Otsu's Method Visualization.gif"></a></p>

Let us recurrent this experiment:


**Show the strength results like paper**


In [ ]:
import numpy as np
img = I.astype(np.float32)
R = img[:, :, 0]
G = img[:, :, 1]
B = img[:, :, 2]
ExG = 2 * G - R - B
ExR = 1.4 * R - G

In [ ]:
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(9, 3), dpi=300)
ax[0].set_title("ExG Tonal Image")
ax[0].imshow(ExG, cmap="gray")

ax[1].set_title("ExR Tonal Image")
ax[1].imshow(ExR, cmap="gray")

plt.show()

In [ ]:
thresh = threshold_otsu(ExG)
bin_img_1 = ExG>thresh
bin_img=bin_img_1.astype(np.int)
bin_img.shape
plt.imshow(bin_img)
plt.title(f"ExG + Otsu")

In [ ]:
bin_img_1[1:5,1:5]

In [ ]:
bin_img[1:5,1:5]

fomular from paper

In [ ]:
ex_GR1 = ExG - ExR
thresh = 0
bin_img_1 = ex_GR1>thresh
bin_img=bin_img_1.astype(np.int)
bin_img.shape
plt.imshow(bin_img)
plt.title(f"ExG -ExR")

In [ ]:
rgbimg = I.astype(np.int)
rgbimg[:, :, 0] = rgbimg[:, :, 0]*bin_img
rgbimg[:, :, 1] = rgbimg[:, :, 1]*bin_img
rgbimg[:, :, 2] = rgbimg[:, :, 2]*bin_img
plt.imshow(rgbimg)
plt.title(f"ROI")

## Run whole folder

In [ ]:
img_list = io.imread_collection("content/soybean_s_c/*.JPG")
img_list.files

Define save path

In [ ]:
savepath = 'soybean_s_c_seg/'
# Check whether the specified path exists or not
isExist = os.path.exists(savepath)
if not isExist:
  # Create a new directory because it does not exist
  os.makedirs(savepath)
  print('Created path for save')

In [ ]:
im_seg_list=[]
for imgname in img_list.files:
    I = io.imread(imgname)
    img = I.astype(np.float32)
    R = img[:, :, 0]
    G = img[:, :, 1]
    B = img[:, :, 2]
    ExG = 2 * G - R - B
    ExR = 1.4 * R - G
    ex_GR1 = ExG - ExR
    thresh = 0
    bin_img_1 = ex_GR1>thresh
    bin_img=bin_img_1.astype(np.uint8)
    rgbimg = I.astype(np.uint8)
    rgbimg[:, :, 0] = rgbimg[:, :, 0]*bin_img
    rgbimg[:, :, 1] = rgbimg[:, :, 1]*bin_img
    rgbimg[:, :, 2] = rgbimg[:, :, 2]*bin_img
    im_seg_list.append(rgbimg)
    c_imgname=savepath+imgname[20:-4]+'_seg.JPG'
    io.imsave(c_imgname,rgbimg)

In [ ]:
fig = io.imshow_collection(im_seg_list);
fig.set_figwidth(20)
fig.set_figheight(10)

In [ ]:
!zip -r /content/soybean_s_c_seg.zip /content/soybean_s_c_seg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copy to google drive to save
import shutil
shutil.copyfile('/content/soybean_s_c_seg.zip', '/content/drive/MyDrive/uktest/soybean_s_c_seg.zip')